In [1]:
# dev of full attack of a model

In [2]:
from __future__ import print_function

In [3]:
import matplotlib as mpl
# set this 'backend' when using jupyter; do this before importing pyplot
mpl.use('nbagg')
import matplotlib.pyplot as plt

In [4]:
import torch
import torch.nn as nn

import numpy as np
from scipy import stats

import sys
sys.path.append('/home/felix/Research/Adversarial Research/FGN---Research/')
import Finite_Gaussian_Network_lib as fgnl
import Finite_Gaussian_Network_lib.fgn_helper_lib as fgnh

In [5]:
!gpustat

countdown  Sat Aug 17 14:29:04 2019
[0] GeForce GTX 1080 | 39'C,   0 % |  1809 /  8119 MB | felix(559M) felix(685M) felix(553M)
[1] GeForce GTX 1080 | 40'C,   0 % |  1437 /  8118 MB | felix(601M)


In [6]:
# Define what device we are using
print("CUDA Available: ",torch.cuda.is_available())
use_cuda = True
device = torch.device("cuda" if (use_cuda and torch.cuda.is_available()) else "cpu")
print("Using device:", device)

# manualy set cuda device
torch.cuda.set_device(1)

CUDA Available:  True
Using device: cuda


In [7]:
# # random seeds
# torch.manual_seed(999)
# np.random.seed(999)

# torch.backends.cudnn.deterministic = True
# torch.cuda.manual_seed_all(999)

In [8]:
# load MNIST data
batch_size = 192
(mnist_train_loader, mnist_val_loader, mnist_test_loader) = fgnh.mnist_dataloaders(batch_size)
# minimum/maximum pixel value post normalization, from train dataset
min_pix = -0.4242129623889923095703125
max_pix =  2.8214867115020751953125000

In [9]:
# load classic model
# model_path = "../Experiments/sample_models/sample_fgn_model_noisy_eval_p0.27_full.pth"
classic_model_path = "../Experiments/sample_models/sample_classic_model_noisy_eval_p0.65_full.pth"
classic_model = torch.load(classic_model_path)
classic_model.to(device)

Feedforward_Classic_net(
  (hidden_layers): ModuleList(
    (0): Dropout(p=0.03125)
    (1): Linear(in_features=784, out_features=32, bias=True)
    (2): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Dropout(p=0.03125)
    (4): Linear(in_features=32, out_features=32, bias=True)
    (5): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): Dropout(p=0.03125)
  )
  (ib): BatchNorm1d(784, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fl): Linear(in_features=32, out_features=10, bias=True)
)

In [10]:
# test the model
classic_model.eval()

# loss func
cross_ent_loss = fgnh.def_classical_cross_ent_loss()

test_res = fgnh.test(classic_model, mnist_train_loader,
                        cross_ent_loss, pred_func=fgnh.cross_ent_pred_accuracy, verbose=True)

test_res = fgnh.test(classic_model, mnist_val_loader,
                        cross_ent_loss, pred_func=fgnh.cross_ent_pred_accuracy, verbose=True)

Test set - Average loss: 0.2577, Accuracy: 46105/50000 (92%)
Test set - Average loss: 0.2488, Accuracy: 9228/10000 (92%)


In [11]:
# load fgn model
fgn_model_path = "../Experiments/sample_models/sample_fgn_model_noisy_eval_p0.27_full.pth"
fgn_model = torch.load(fgn_model_path)
fgn_model.to(device)

Feedforward_FGN_net(
  (hidden_layers): ModuleList(
    (0): Dropout(p=0.03125)
    (1): FGN_layer()
    (2): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Dropout(p=0.03125)
    (4): FGN_layer()
    (5): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): Dropout(p=0.03125)
  )
  (ib): BatchNorm1d(784, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fl): FGN_layer()
)

In [12]:
# test the model
fgn_model.eval()
# switch to random eval
try:
    fgn_model.set_random_eval(True)
except: 
    pass

# loss func
cross_ent_loss = fgnh.def_classical_cross_ent_loss()

test_res = fgnh.test(fgn_model, mnist_train_loader,
                        cross_ent_loss, pred_func=fgnh.cross_ent_pred_accuracy, verbose=True)

test_res = fgnh.test(fgn_model, mnist_val_loader,
                        cross_ent_loss, pred_func=fgnh.cross_ent_pred_accuracy, verbose=True)

Test set - Average loss: 0.2871, Accuracy: 45284/50000 (91%)
Test set - Average loss: 0.2775, Accuracy: 9100/10000 (91%)


In [13]:
### Beginning of attacks - untargeted

In [14]:
# parameters of the experiment
attack_params = {'targeted':False, 
                 'data_classes':range(10), 
                 'max_noise':(max_pix-min_pix)/16.0, 
                 'data_bounds':fgnh.get_dataloader_bounds(mnist_train_loader),
                 'loss_func':torch.nn.CrossEntropyLoss(), 
                 'step_size':(max_pix-min_pix)/128.0, 
                 'steps': 5,
                 'confidence_req':0.5}

num_attacks = len(mnist_val_loader.dataset.tensors[0])
# num_attacks = 100
verbose = False

In [15]:
# attack the classic net
classic_untargeted_attack_results = fgnl.full_FGSM_attack(classic_model, mnist_val_loader, num_attacks=num_attacks, 
                                                          attack_params = attack_params, verbose=verbose)

classic_untargeted_confidence_dist = classic_untargeted_attack_results['confidence_dist']
classic_untargeted_successful_attack_count = classic_untargeted_attack_results['successful_attack_count']
classic_untargeted_steps_dist = classic_untargeted_attack_results['steps_dist']
classic_untargeted_adv_class_dist = classic_untargeted_attack_results['adv_class_dist']

In [ ]:
# attack the fgn net
fgn_untargeted_attack_results = fgnl.full_FGSM_attack(fgn_model, mnist_val_loader, num_attacks=num_attacks,
                                                      attack_params = attack_params, verbose=False)

fgn_untargeted_confidence_dist = fgn_untargeted_attack_results['confidence_dist']
fgn_untargeted_successful_attack_count = fgn_untargeted_attack_results['successful_attack_count']
fgn_untargeted_steps_dist = fgn_untargeted_attack_results['steps_dist']
fgn_untargeted_adv_class_dist = fgn_untargeted_attack_results['adv_class_dist']

In [ ]:
### results

In [ ]:
# success rate
print("attack on classic net success rate: {}/{}, {}".format(classic_untargeted_successful_attack_count, num_attacks, 
                                                             float(classic_untargeted_successful_attack_count)/num_attacks))
print("attack on fgn success rate: {}/{}, {}".format(fgn_untargeted_successful_attack_count, num_attacks, 
                                                     float(fgn_untargeted_successful_attack_count)/num_attacks))

In [ ]:
# plot confidence distribution
striped_classic_confidence = [x for x in classic_untargeted_confidence_dist if x>=0]
plt.hist(striped_classic_confidence, bins=100, alpha=0.5, label='classic')
print("classic confidence stats:\n", stats.describe(striped_classic_confidence))

striped_fgn_confidence = [x for x in fgn_untargeted_confidence_dist if x>=0]
plt.hist(striped_fgn_confidence, bins=100, alpha=0.5, label='fgn')
print("fgn confidence stats:\n", stats.describe(striped_fgn_confidence))

plt.legend()
plt.show()

In [ ]:
# plot steps distribution
plt.hist(classic_untargeted_steps_dist, bins=range(attack_params['steps']+2), align='left',rwidth =0.8, alpha=0.5, label='classic')
plt.hist(fgn_untargeted_steps_dist, bins=range(attack_params['steps']+2), align='left',rwidth =0.8, alpha=0.5, label='fgn')

plt.legend()
plt.show()

In [ ]:
# targeted attacks

In [ ]:
# change params for targeted attack
attack_params = {'targeted':True, 
                 'data_classes':range(10), 
                 'max_noise':(max_pix-min_pix)/64.0, 
                 'data_bounds':fgnh.get_dataloader_bounds(mnist_train_loader),
                 'loss_func':torch.nn.CrossEntropyLoss(), 
                 'step_size':(max_pix-min_pix)/128.0, 
                 'steps': 5,
                 'confidence_req':0.5}

# num_attacks = len(mnist_val_loader.dataset.tensors[0])
# num_attacks = 10

In [ ]:
# attack the classic net
classic_targeted_attack_results = fgnl.full_FGSM_attack(classic_model, mnist_val_loader, num_attacks=num_attacks, 
                                                          attack_params = attack_params, verbose=verbose)

classic_targeted_confidence_dist = classic_targeted_attack_results['confidence_dist']
classic_targeted_successful_attack_count = classic_targeted_attack_results['successful_attack_count']
classic_targeted_steps_dist = classic_targeted_attack_results['steps_dist']
classic_targeted_adv_class_dist = classic_targeted_attack_results['adv_class_dist']

In [ ]:
# attack the fgn net
fgn_targeted_attack_results = fgnl.full_FGSM_attack(fgn_model, mnist_val_loader, num_attacks=num_attacks,
                                                      attack_params = attack_params, verbose=verbose)

fgn_targeted_confidence_dist = fgn_targeted_attack_results['confidence_dist']
fgn_targeted_successful_attack_count = fgn_targeted_attack_results['successful_attack_count']
fgn_targeted_steps_dist = fgn_targeted_attack_results['steps_dist']
fgn_targeted_adv_class_dist = fgn_targeted_attack_results['adv_class_dist']

In [ ]:
# success rate
print("attack on classic net success rate: {}/{}, {}".format(classic_targeted_successful_attack_count, num_attacks, 
                                                             float(classic_targeted_successful_attack_count)/num_attacks))
print("attack on fgn success rate: {}/{}, {}".format(fgn_targeted_successful_attack_count, num_attacks, 
                                                     float(fgn_targeted_successful_attack_count)/num_attacks))

In [ ]:
# plot confidence distribution
striped_classic_confidence = [x for x in classic_targeted_confidence_dist if x>=0]
plt.hist(striped_classic_confidence, bins=100, alpha=0.5, label='classic')
print("classic confidence stats:\n", stats.describe(striped_classic_confidence))

striped_fgn_confidence = [x for x in fgn_targeted_confidence_dist if x>=0]
plt.hist(striped_fgn_confidence, bins=100, alpha=0.5, label='fgn')
print("fgn confidence stats:\n", stats.describe(striped_fgn_confidence))

plt.legend()
plt.show()

In [ ]:
# plot steps distribution
plt.hist(classic_targeted_steps_dist, bins=range(attack_params['steps']+2), align='left',rwidth =0.8, alpha=0.5, label='classic')
plt.hist(fgn_targeted_steps_dist, bins=range(attack_params['steps']+2), align='left',rwidth =0.8, alpha=0.5, label='fgn')

plt.legend()
plt.show()